<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests
!pip install html5lib
!pip install bs4

In [4]:
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import requests
from bs4 import BeautifulSoup
  

In [11]:
import sys
 
# the setrecursionlimit function is
# used to modify the default recursion
# limit set by python. Using this,
# we can increase the recursion limit
# to satisfy our needs
 
sys.setrecursionlimit(10**6)

In [12]:
visited = {}

def go_to_character_page(URL):
  if(URL not in visited):
    print("cPage: "+URL)
    
    r = requests.get(URL)
    
    soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
    name = soup.find('div', attrs = {'class':'meta__title'}).h1.text
    visited[URL] = {'name':name}
    
    film = soup.find('ul', attrs = {'class':'meta__stats'}).text.strip()
    visited[URL]['film'] = film

    attr_html = soup.find('div', attrs = {'class':'character-profile__summary'}).div.find_all_next('p')
    for a in attr_html:
      if(a.strong != None):
        attr = a.strong.text.replace('…','').replace('.','').replace('\n','').strip().lower()
        value = (a.text.replace(attr, '').strip())
        # print(attr, value)
        visited[URL][attr] = value

    #go to characters like this one
    buttons = soup.find_all('a', attrs = {'class':'button'})
    urls = ["https://www.charactour.com"+b['href'] for b in buttons if ('/hub/characters/characters-like' in str(b))]
    visited[URL]['like'] = urls
    for url in urls:
      go_to_list_page(url)
        
def go_to_list_page(URL):
  if(URL not in visited):
    visited[URL] = None
    print("lPage:"+URL)

    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')

    #find all characters listed in the page
    attr_html = soup.find_all('a', attrs = {'class':'results-label dotdotdot'})
    for a in attr_html:
      p = 'https://www.charactour.com'+a['href']
      go_to_character_page(p)

    #find all the paginated pages
    attr_html = soup.find('ul', attrs = {'class':'pagination'})
    if attr_html is not None:
      pages = [li.a['href']for li in attr_html  if ('https://www.charactour.com/hub' in str(li)) ]
      for p in pages:
          go_to_list_page(p)


In [ ]:
go_to_list_page("https://www.charactour.com/hub/recs/popular-characters")

In [14]:
import pandas as pd
new_list = []
for key in visited:
   if(visited[key]!=None):
     visited[key]['url'] = key
     new_list.append(visited[key])
df = pd.DataFrame(new_list)
# print([visited[key] for key in visited if(visited[key]!=None)])
df

,name,film,grew up,living,profession,interests,relationship status,challenge,personality,like,url,overview,visiting,see him in,see her in,growing up,personality “,professional,relationships,,challenges,professions,interests include,grew u,relationship,elena,stefan,damon,the first book in a new trilogy from l j smith!,"living in beacon hills, california, with his mom beacon hills seems like a completely normal town, full of totally average people however, the werewolf apparently roaming the woods around town suggests that things are about to get a lot weirder",profession “,lives,interest,living in,friends
0,Hermione Granger,Harry Potter Series,"Grew Up… in a stable, loving household in Lond...","Living… at the Hogwarts castle, in Gryffindor ...","Profession… student at Hogwarts, working on ma...","Interests… studying, studying and more studyin...","Relationship Status… single. She’s only 11, pl...",Challenge… helping Harry and Ron while keeping...,"Personality… studious, driven, organized, and ...",[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marlena Rosenbluth,Water for Elephants,"Grew Up… an orphan, passed around from home to...","Living… on the road, as part of the Benzini Br...",Profession… performer. She is the Benzini Brot...,Interests… taking care of her animals. Marlena...,Relationship Status… married to August Rosenbl...,Challenge… deciding which way to go. Marlena h...,"Personality… gentle, dutiful, and, ethereal. M...",[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kathy,Never Let Me Go,"Grew Up... at Hailsham, a boarding school in t...",NaN,NaN,NaN,NaN,NaN,NaN,[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Robbie Turner,Atonement,NaN,NaN,NaN,NaN,NaN,NaN,"Personality... dashing, educated, and clumsy. ...",[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,Overview... the son of an English servant work...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anna Karenina,Anna Karenina,NaN,Living... in the late 1800s with her husband A...,Profession... stay-at-home mother. Anna deeply...,Interests... books. Anna is a voracious reader...,Relationship Status... married to Alexei Karen...,Challenge... enduring the fallout after tellin...,"Personality... elegant, intelligent, and stron...",[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,Lexie Grey,Grey's Anatomy,Grew Up... happy and popular. Lexie was raised...,"Living... in Seattle, Washington. She moved ba...",Profession... surgical intern at Seattle Grace...,"Interests... binge eating when stressed, putti...",Relationship Status... single. There are more ...,Challenge... getting to know her sister. Lexie...,"Personality... optimistic, empathetic, vulnera...",[https://www.charactour.com/hub/characters/cha...,https://www.charactour.com/hub/characters/view...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1303,Rick Sanchez,Rick and Morty,NaN,Living… across an infinite amount of parallel ...,Profession… mad scientist. He travels into dre...,Interests… exploring inter-dimensional space a...,Relationship Status… divorced and widowed. Ric...,Challenge… keeping himself occupied. Rick is c...,Personality…

In [15]:
df.to_csv("/content/drive/MyDrive/ANLP21/character_tour.csv")

In [17]:
from google.colab import files

files.download("/content/drive/MyDrive/ANLP21/character_tour.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>